In [1]:
pip install pandas==1.2.4
pip install numpy==1.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 11.5 MB/s eta 0:00:0000:01:01m
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
# importing libraries
import pandas as pd
import numpy as np

In [2]:
# checking the version 
print(pd.__version__)
print(np.__version__)

1.2.4
1.20.3


In [3]:
# laoding dataframes
diagnosis_df=pd.read_csv("diagnosis.csv")
patient_df = pd.read_csv("patient.csv")
visit_df = pd.read_csv("visit.csv")

In [44]:
diagnosis_df.head()

,person_id,diagnosis_date,diagnosis_id,diagnosis
0,10107,2016-04-06,54245,D1.2
1,10492,2017-05-18,54500,D1.2
2,11223,2021-06-24,58782,C1.1
3,10009,2017-05-24,59462,D1.2
4,10671,2016-06-21,62066,D1.2


In [45]:
patient_df.head()

,person_id,date_of_birth,gender
0,10000,1954-03-17,F
1,10001,1963-01-06,F
2,10002,1961-11-17,M
3,10003,1956-04-24,F
4,10004,1963-07-04,F


In [46]:
visit_df.head()

,unit_name,visit_id,visit_date,test_name,test_value,person_id
0,Unit02,53816,2021-01-05,TestA,64.104983,11476
1,Unit01,54189,2016-07-18,TestC,68.506593,11134
2,Unit02,54406,2017-07-21,TestA,50.432863,10154
3,Unit02,56053,2019-09-10,TestA,74.358531,10261
4,Unit02,56740,2021-01-20,TestA,39.958341,11282


In [47]:
diagnosis_null_count=diagnosis_df.isnull().sum()
diagnosis_null_count

person_id         0
diagnosis_date    0
diagnosis_id      0
diagnosis         0
dtype: int64

In [48]:
visit_null_count=visit_df.isnull().sum()
visit_null_count

unit_name     0
visit_id      0
visit_date    0
test_name     0
test_value    0
person_id     0
dtype: int64

In [49]:
patient_null_count=patient_df.isnull().sum()
patient_null_count

person_id        0
date_of_birth    0
gender           0
dtype: int64

In [10]:
visit_df.shape

(11346, 6)

In [11]:
diagnosis_df.shape

(3782, 4)

In [54]:
# condition-based filtering for 'Unit01' and 'D1.2'
filtered_visit = visit_df[visit_df['unit_name'] == 'Unit01']
filtered_diagnosis = diagnosis_df[diagnosis_df['diagnosis'] == "D1.2" ]
#filtered_df = np.where(diagnosis_df['diagnosis'] == 'D1.2')

In [56]:
filtered_diagnosis.shape

(2782, 4)

In [57]:
filtered_visit.shape


(3315, 6)

In [27]:
filtered_visit.head()

,unit_name,visit_id,visit_date,test_name,test_value,person_id
1,Unit01,54189,2016-07-18,TestC,68.506593,11134
7,Unit01,59134,2017-06-27,TestC,27.982604,11767
9,Unit01,59944,2016-02-09,TestB,25.064054,11535
10,Unit01,61221,2021-03-29,TestC,97.890644,10179
13,Unit01,62350,2017-06-28,TestB,39.138280,11677


In [29]:
# converting to date columns to DateTime format
filtered_diagnosis['diagnosis_date'] = pd.to_datetime(diagnosis_df['diagnosis_date'])
filtered_visit['visit_date'] = pd.to_datetime(filtered_visit['visit_date'])

/var/folders/wr/prwm0mbx7m98xhz8f24wmh5r0000gn/T/ipykernel_5075/499807394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_diagnosis['diagnosis_date'] = pd.to_datetime(diagnosis_df['diagnosis_date'])
/var/folders/wr/prwm0mbx7m98xhz8f24wmh5r0000gn/T/ipykernel_5075/499807394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_visit['visit_date'] = pd.to_datetime(filtered_visit['visit_date'])


In [30]:
filtered_diagnosis.dtypes

person_id                  int64
diagnosis_date    datetime64[ns]
diagnosis_id               int64
diagnosis                 object
dtype: object

In [22]:
# creating a merge_df from filtered_df
merge_df = pd.merge(filtered_visit, filtered_diagnosis, on='person_id', how='inner')
# Condition to check visiti dates falls within the 7 days of diagnosis date
merge_df = merge_df[(merge_df['visit_date'] > merge_df['diagnosis_date']) &
                    (merge_df['visit_date'] <= merge_df['diagnosis_date'] + pd.DateOffset(days=7))]


In [40]:
# sorting the values by person_id
merge_df.sort_values(by='person_id', ascending= True, inplace=True)

In [41]:
merge_df.head()

,unit_name,visit_id,visit_date,test_name,test_value,person_id,diagnosis_date,diagnosis_id,diagnosis
7524,Unit01,3861345,2021-02-28,TestC,37.019023,10008,2021-02-26,2180352,D1.2
2034,Unit01,1616367,2017-04-09,TestC,47.570295,10009,2017-04-07,4585470,D1.2
2026,Unit01,605187,2017-05-27,TestB,72.814277,10009,2017-05-24,59462,D1.2
2037,Unit01,4386690,2016-03-29,TestC,48.975403,10009,2016-03-25,1159902,D1.2
2043,Unit01,7879515,2017-05-09,TestB,80.845555,10009,2017-05-07,2151709,D1.2


In [58]:
# finding out the unique_id in person_id column to check number of patient_id 
len(merge_df['person_id'].unique())
print("Number of uniqie patients : ",len(merge_df['person_id'].unique()))

Number of uniqie patients :  800


In [43]:
result_df

,person_id,Diagnose_date,Visit_date
1304,10008,2021-02-26,2021-02-28
6,10009,2017-05-24,2017-05-27
7,10009,2016-03-25,2016-03-29
8,10009,2017-05-07,2017-05-09
9,10009,2017-04-07,2017-04-09
...,...,...,...
1130,11996,2017-05-23,2017-05-25
1128,11996,2017-01-10,2017-01-14
1129,11996,2016-05-20,2016-05-21
1695,11998,2017-05-17,2017-05-20


In [28]:
## different approach 
result_df = pd.DataFrame(columns = ['person_id','Diagnose_date','Visit_date'])
num_patients=0
for person_id in filtered_diagnosis['person_id'].unique():
  person_visit = filtered_visit[filtered_visit['person_id'] == person_id ]
  person_diagnosis = filtered_diagnosis[filtered_diagnosis['person_id'] == person_id]
  for _,diag_row in person_diagnosis.iterrows():
    for _,visit_row in person_visit.iterrows():
      if visit_row['visit_date'] > diag_row['diagnosis_date'] and visit_row['visit_date'] <= diag_row['diagnosis_date'] + pd.DateOffset(days=7):
        result_df = result_df.append( { 'person_id' : visit_row['person_id'],
                                         'Diagnose_date' : diag_row['diagnosis_date'],
                                          'Visit_date' : visit_row['visit_date']}, ignore_index=True
        )
        num_patients+=1

In [ ]:
print("Number of uniqie patients : ",len(result_df['person_id'].unique()))

In [ ]:
result_df.to_csv("result.csv")

In [ ]:
result_df.sort_values(by='person_id', ascending= True, inplace=True)

In [59]:
result_df.head()

,person_id,Diagnose_date,Visit_date
1304,10008,2021-02-26,2021-02-28
6,10009,2017-05-24,2017-05-27
7,10009,2016-03-25,2016-03-29
8,10009,2017-05-07,2017-05-09
9,10009,2017-04-07,2017-04-09
